In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
import re
import scipy.stats as st
import matplotlib.pyplot as plt


In [2]:
raw_data = pd.read_csv('all-weeks-countries.csv')
raw_data.head(5000)

,country_name,country_iso2,week,category,weekly_rank,show_title,season_title,cumulative_weeks_in_top_10
0,Argentina,AR,2022-01-09,Films,1,Don't Look Up,NaN,3
1,Argentina,AR,2022-01-09,Films,2,Mother/Android,NaN,1
2,Argentina,AR,2022-01-09,Films,3,The Lost Daughter,NaN,2
3,Argentina,AR,2022-01-09,Films,4,Backtrace,NaN,1
4,Argentina,AR,2022-01-09,Films,5,Four to Dinner,NaN,1
...,...,...,...,...,...,...,...,...
4995,Brazil,BR,2021-07-18,TV,6,Sex/Life,Sex/Life: Season 1,3
4996,Brazil,BR,2021-07-18,TV,7,Elize Matsunaga: Once Upon a Crime,Elize Matsunaga: Once Upon a Crime: Season 1,2
4997,Brazil,BR,2021-07-18,TV,8,As Aventuras de Poliana,As Aventuras de Poliana: Season 1,3
4998,Brazil,BR,2021-07-18,TV,9,Atypical,Atypical: Season 4,2


In [3]:
raw_data['season_title'].unique()

array([nan, 'Café con aroma de mujer: Season 1', 'Cobra Kai: Season 4',
       'The Queen of Flow: Season 2', 'Stay Close: Limited Series',
       'The Queen of Flow: Season 1', 'Emily in Paris: Season 2',
       'Pasión de Gavilanes: Season 1', 'The Witcher: Season 2',
       'Rebelde: Season 1', 'Rebelde Way: Temporada 1',
       'Daughter From Another Mother: Season 2',
       'The Girl from Oslo: Season 1', 'Money Heist: Part 5',
       'The Witcher: Season 1', 'Yo soy Betty, la fea: Season 1',
       'Titans: Season 3', 'Pablo Escobar, el patrón del mal: Season 1',
       'Lost in Space: Season 3',
       'Jurassic World Camp Cretaceous: Season 4', 'Maid: Limited Series',
       'Elves: Season 1', 'Hellbound: Season 1',
       'True Story: Limited Series', 'Arcane: Season 1',
       'Squid Game: Season 1', 'You: Season 3',
       'Narcos: Mexico: Season 3', 'Luis Miguel - The Series: Season 3',
       'Where is Marta?: Limited Series', 'The Five Juanas: Season 1',
       'Maya and

In [4]:
films = raw_data[raw_data['season_title'].isnull()]
films.head()

,country_name,country_iso2,week,category,weekly_rank,show_title,season_title,cumulative_weeks_in_top_10
0,Argentina,AR,2022-01-09,Films,1,Don't Look Up,NaN,3
1,Argentina,AR,2022-01-09,Films,2,Mother/Android,NaN,1
2,Argentina,AR,2022-01-09,Films,3,The Lost Daughter,NaN,2
3,Argentina,AR,2022-01-09,Films,4,Backtrace,NaN,1
4,Argentina,AR,2022-01-09,Films,5,Four to Dinner,NaN,1


In [5]:
series = raw_data.dropna()
series

,country_name,country_iso2,week,category,weekly_rank,show_title,season_title,cumulative_weeks_in_top_10
10,Argentina,AR,2022-01-09,TV,1,Café con aroma de mujer,Café con aroma de mujer: Season 1,2
11,Argentina,AR,2022-01-09,TV,2,Cobra Kai,Cobra Kai: Season 4,2
12,Argentina,AR,2022-01-09,TV,3,The Queen of Flow,The Queen of Flow: Season 2,8
13,Argentina,AR,2022-01-09,TV,4,Stay Close,Stay Close: Limited Series,1
14,Argentina,AR,2022-01-09,TV,5,The Queen of Flow,The Queen of Flow: Season 1,10
...,...,...,...,...,...,...,...,...
52635,Vietnam,VN,2021-07-04,TV,6,Reply 1988,Reply 1988: Season 1,1
52636,Vietnam,VN,2021-07-04,TV,7,"Nevertheless,","Nevertheless,: Season 1",1
52637,Vietnam,VN,2021-07-04,TV,8,Too Hot to Handle,Too Hot to Handle: Season 2,1
52638,Vietnam,VN,2021-07-04,TV,9,Record of Ragnarok,Record of Ragnarok: Season 1,1


In [8]:
series['Week Number'] = series['week'].dt.strptime('%Y-%m-%d')

AttributeError: Can only use .dt accessor with datetimelike values